## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [2]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="ethereum"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('ethereum_train_7_3.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,0.23180,0.24981,0.23085,0.24797,11970598,USD
1,2020-06-02,0.24801,0.25321,0.22795,0.23794,12686011,USD
2,2020-06-03,0.23790,0.24476,0.23384,0.24452,8151002,USD
3,2020-06-04,0.24453,0.24641,0.23668,0.24337,8749785,USD
4,2020-06-05,0.24337,0.24750,0.23970,0.24023,7754761,USD
...,...,...,...,...,...,...,...
507,2021-10-21,4.16051,4.36691,4.02948,4.05416,1140074,USD
508,2021-10-22,4.05389,4.16411,3.89640,3.97094,654771,USD
509,2021-10-23,3.97094,4.16933,3.93879,4.16885,487025,USD
510,2021-10-24,4.16880,4.18400,3.96584,4.08280,507629,USD


Chuẩn hóa dữ liệu

In [3]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Open'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Open'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Open'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Open'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Open'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}7_3_standardized.csv")
print(df)

           Date     Open     High      Low    Close   Volume Currency  \
20   2020-06-21  0.22887  0.23118  0.22738  0.22813  4102533      USD   
21   2020-06-22  0.22811  0.24639  0.22769  0.24325  7929329      USD   
22   2020-06-23  0.24324  0.24477  0.24094  0.24320  5163349      USD   
23   2020-06-24  0.24320  0.24923  0.23156  0.23475  9127950      USD   
24   2020-06-25  0.23475  0.23529  0.22799  0.23256  6475813      USD   
..          ...      ...      ...      ...      ...      ...      ...   
507  2021-10-21  4.16051  4.36691  4.02948  4.05416  1140074      USD   
508  2021-10-22  4.05389  4.16411  3.89640  3.97094   654771      USD   
509  2021-10-23  3.97094  4.16933  3.93879  4.16885   487025      USD   
510  2021-10-24  4.16880  4.18400  3.96584  4.08280   507629      USD   
511  2021-10-25  4.08268  4.23476  4.06994  4.21992   481414      USD   

         H-L      O-C     SMA_7    SMA_14    SMA_21      SD_7     SD_21  
20   0.00380  0.00074  0.231636  0.236451  0.2380

Scale Miền giá trị lại

In [4]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Open']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[0.00197341],
       [0.00178113],
       [0.00560905],
       [0.00559893],
       [0.00346107],
       [0.00290952],
       [0.00218847],
       [0.        ],
       [0.00101707],
       [0.00172294],
       [0.00114863],
       [0.00250219],
       [0.00135356],
       [0.00101707],
       [0.00202907],
       [0.00174065],
       [0.00518401],
       [0.0046021 ],
       [0.00654263],
       [0.00529786],
       [0.00509546],
       [0.00458692],
       [0.00550785],
       [0.00472607],
       [0.0049133 ],
       [0.00444018],
       [0.00318276],
       [0.00296012],
       [0.00370648],
       [0.00454391],
       [0.00380262],
       [0.00619096],
       [0.01068174],
       [0.01372788],
       [0.01473989],
       [0.02131541],
       [0.02278282],
       [0.02548994],
       [0.02425782],
       [0.02447034],
       [0.02878402],
       [0.03172896],
       [0.04201855],
       [0.03825135],
       [0.04169218],
       [0.04264346],
       [0.04547203],
       [0.043

In [5]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [6]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}73.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 14s 42ms/step - loss: 0.0311
Epoch 2/60
40/40 [==============================] - 2s 46ms/step - loss: 0.0109
Epoch 3/60
40/40 [==============================] - 2s 43ms/step - loss: 0.0096
Epoch 4/60
40/40 [==============================] - 2s 48ms/step - loss: 0.0071
Epoch 5/60
40/40 [==============================] - 2s 48ms/step - loss: 0.0092
Epoch 6/60
40/40 [==============================] - 2s 48ms/step - loss: 0.0070
Epoch 7/60
40/40 [==============================] - 2s 50ms/step - loss: 0.0076
Epoch 8/60
40/40 [==============================] - 2s 50ms/step - loss: 0.0065
Epoch 9/60
40/40 [==============================] - 2s 49ms/step - loss: 0.0045
Epoch 10/60
40/40 [==============================] - 2s 53ms/step - loss: 0.0062
Epoch 11/60
40/40 [==============================] - 2s 49ms/step - loss: 0.0050
Epoch 12/60
40/40 [==============================] - 2s 54ms/step - loss: 0.0051
Epoch 13/60
40/40 [=================